<a href="https://colab.research.google.com/github/acherm/SEAI4Sport/blob/main/LLM_repeat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import os

# ⚠️ Remplace "TON_API_KEY_ICI" par ta clé Groq :
os.environ["GROQ_API_KEY"] = "" # https://console.groq.com/keys


In [13]:
import os
import csv
import time
import hashlib
from datetime import datetime

import requests
import pandas as pd  # 👈 nouveau

# Constantes "globales" configurables en éditant ce bloc
GROQ_BASE_URL = "https://api.groq.com/openai/v1/chat/completions"
SLEEP_SECONDS_BETWEEN_CALLS = 0.0   # mets 0.1 ou 0.2 si tu veux espacer les requêtes
OUTPUT_DIR = "."                    # ou par ex. "/content/drive/MyDrive/groq_results"

# max_tokens : Détermine la longueur maximale de la réponse.
# temperature : Contrôle la créativité des réponses (0 pour des réponses déterministes, 1 pour des réponses plus créatives).
# top_p : Ajuste le niveau de variation dans le choix des mots.

def run_sensitivity_experiment(
    prompt: str,
    model: str = "llama-3.1-8b-instant",
    temperature: float = 0.7,
    max_tokens: int = 50,
    n_repeats: int = 10,
):
    """
    Appelle un LLM Groq plusieurs fois avec les mêmes paramètres
    et enregistre les réponses dans un CSV.

    Chaque appel à cette fonction crée un nouveau CSV dont le nom contient :
    - la date/heure actuelle
    - le nom du modèle
    - un hash des paramètres (prompt, temperature, max_tokens, model)

    Le CSV contient une ligne par appel :
    - model_name
    - temperature
    - max_tokens
    - prompt
    - answer
    - run_index (0..n_repeats-1)

    Retourne
    -------
    filepath : str
        Chemin du fichier CSV créé.
    df : pandas.DataFrame
        DataFrame contenant les résultats.
    """

    # Clé API : uniquement via l'environnement
    api_key = os.getenv("GROQ_API_KEY")
    if not api_key:
        raise ValueError(
            "API key is missing. Set GROQ_API_KEY in the environment (voir la cellule 1)."
        )

    # Construction d'un hash stable des paramètres expérimentaux
    hash_input = f"{model}||{temperature}||{max_tokens}||{prompt}".encode("utf-8")
    short_hash = hashlib.sha1(hash_input).hexdigest()[:8]

    # Timestamp pour le nom de fichier
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Nettoyage du nom du modèle pour éviter les caractères problématiques
    safe_model_name = "".join(c if c.isalnum() or c in "-_" else "_" for c in model)

    # Nom du fichier CSV
    filename = f"{timestamp}_{safe_model_name}_{short_hash}.csv"
    filepath = os.path.join(OUTPUT_DIR, filename)

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}",
    }

    # Création du dossier de sortie si nécessaire
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # On garde les résultats en mémoire pour construire le DataFrame
    rows = []

    for i in range(n_repeats):
        payload = {
            "model": model,
            "messages": [
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
            "temperature": temperature,
            "max_tokens": max_tokens,
        }

        try:
            response = requests.post(GROQ_BASE_URL, json=payload, headers=headers)
        except requests.RequestException as e:
            answer = f"REQUEST_ERROR: {e}"
        else:
            if response.status_code == 200:
                try:
                    json_resp = response.json()
                    answer = json_resp["choices"][0]["message"]["content"]
                except (KeyError, ValueError) as e:
                    answer = f"PARSE_ERROR: {e} | RAW: {response.text[:200]}"
            else:
                answer = f"HTTP_{response.status_code}: {response.text[:200]}"

        rows.append(
            {
                "model_name": model,
                "temperature": temperature,
                "max_tokens": max_tokens,
                "prompt": prompt,
                "answer": answer,
                "run_index": i,
            }
        )

        if SLEEP_SECONDS_BETWEEN_CALLS > 0:
            time.sleep(SLEEP_SECONDS_BETWEEN_CALLS)

    # Construction du DataFrame
    df = pd.DataFrame(rows)

    # Sauvegarde en CSV
    df.to_csv(filepath, index=False, encoding="utf-8")

    return filepath, df


In [19]:
prompt = "Faut-il nécessairement manger avant d'aller courir le matin ? Répond uniquement oui ou non"

csv_path, df = run_sensitivity_experiment(
    prompt=prompt,
    model="llama-3.1-8b-instant",
    temperature=0.8,
    max_tokens=10,
    n_repeats=20,
)

print("CSV enregistré dans :", csv_path)
df


CSV enregistré dans : ./20251202_091256_llama-3_1-8b-instant_7ec64e7c.csv


,model_name,temperature,max_tokens,prompt,answer,run_index
0,llama-3.1-8b-instant,0.8,10,Faut-il nécessairement manger avant d'aller co...,Non,0
1,llama-3.1-8b-instant,0.8,10,Faut-il nécessairement manger avant d'aller co...,Non,1
2,llama-3.1-8b-instant,0.8,10,Faut-il nécessairement manger avant d'aller co...,Non,2
3,llama-3.1-8b-instant,0.8,10,Faut-il nécessairement manger avant d'aller co...,Oui,3
4,llama-3.1-8b-instant,0.8,10,Faut-il nécessairement manger avant d'aller co...,Non.,4
5,llama-3.1-8b-instant,0.8,10,Faut-il nécessairement manger avant d'aller co...,Non.,5
6,llama-3.1-8b-instant,0.8,10,Faut-il nécessairement manger avant d'aller co...,Non.,6
7,llama-3.1-8b-instant,0.8,10,Faut-il nécessairement manger avant d'aller co...,Non.,7
8,llama-3.1-8b-instant,0.8,10,Faut-il nécessairement manger avant d'aller co...,Non.,8
9,llama-3.1-8b-instant,0.8,10,Faut-il nécessairement manger avant d'aller co...,Non.,9


In [20]:
# Sur une échelle de 1 (absolument non) à 5 (absolument oui), faut-il nécessairement manger avant d'aller courir le matin ?

In [23]:
prompt = "Sur une échelle de 1 (absolument non) à 5 (absolument oui), faut-il nécessairement manger avant d'aller courir le matin ? Répond uniquement par 1, 2, 3, 4, ou 5"

csv_path, df = run_sensitivity_experiment(
    prompt=prompt,
    model="llama-3.1-8b-instant",
    temperature=0.8,
    max_tokens=10,
    n_repeats=20,
)

print("CSV enregistré dans :", csv_path)
df

CSV enregistré dans : ./20251202_091704_llama-3_1-8b-instant_821213e4.csv


,model_name,temperature,max_tokens,prompt,answer,run_index
0,llama-3.1-8b-instant,0.8,10,Sur une échelle de 1 (absolument non) à 5 (abs...,3,0
1,llama-3.1-8b-instant,0.8,10,Sur une échelle de 1 (absolument non) à 5 (abs...,2,1
2,llama-3.1-8b-instant,0.8,10,Sur une échelle de 1 (absolument non) à 5 (abs...,2,2
3,llama-3.1-8b-instant,0.8,10,Sur une échelle de 1 (absolument non) à 5 (abs...,2,3
4,llama-3.1-8b-instant,0.8,10,Sur une échelle de 1 (absolument non) à 5 (abs...,3,4
5,llama-3.1-8b-instant,0.8,10,Sur une échelle de 1 (absolument non) à 5 (abs...,3,5
6,llama-3.1-8b-instant,0.8,10,Sur une échelle de 1 (absolument non) à 5 (abs...,3,6
7,llama-3.1-8b-instant,0.8,10,Sur une échelle de 1 (absolument non) à 5 (abs...,4,7
8,llama-3.1-8b-instant,0.8,10,Sur une échelle de 1 (absolument non) à 5 (abs...,3,8
9,llama-3.1-8b-instant,0.8,10,Sur une échelle de 1 (absolument non) à 5 (abs...,3,9


In [ ]:
 # "What will be the women's 800 meter run world record by the year 2025?",
 #           "What will be the women's 800 meter run world record by the year 2028?",
 #           "What will be the women's 800 meter run world record by the year 2032?",
 #           "What will be the women's 800 meter run world record by the year 2036?",
 #           "What will be the women's 800 meter run world record by the year 2040?"

# Montrer la sensibilité des LLMs, en répétant les réponses à des prompts, sur des questions de santé/performance 

# Montrer la robustesse des LLMs, en répétant les réponses à des prompts, sur des questions de santé/performance 

# Montrer la cohérence ou la non-cohérence des LLMs sur des prédictions dans le temps… demander, par exemple, la prévision du record du monde de 800m féminin en 2026, 2030, 2040, 2050.


In [1]:
 # "What will be the women's 800 meter run world record by the year 2025?",
 #           "What will be the women's 800 meter run world record by the year 2028?",
 #           "What will be the women's 800 meter run world record by the year 2032?",
 #           "What will be the women's 800 meter run world record by the year 2036?",
 #           "What will be the women's 800 meter run world record by the year 2040?"